# Create data for next steps

This notebook will create a .pkl file with the model's preditions and other information.

In [ ]:
import os
import gc
import pickle
import numpy as np
import torch
import pandas as pd
from torch.utils.data import DataLoader
from tqdm import tqdm

from utils.utils import load_config
from dataloaders import *  
from models import *       
from uncertainty.MCDropout import MCDropout
from validate import validation_metrics, calibration_metrics

## Constants

In [ ]:
DEVICE = 'cuda'
BATCH_SIZE = 64
POSITIVE_THRESHOLD = 0.5

## Experiment Settings

"_LS_01", "_LS_03", "_LS_05", "_LINEAR", "_SIGMOID", "_STEP"

In [ ]:
EXPERIMENTS_FOLDER = ''  # change experiment folder here
MODEL_NAME ='VGGFace_FINAL'  # change model here
BASE_EXPERIMENT_PATH = os.path.join('experiments', EXPERIMENTS_FOLDER, MODEL_NAME)

### MCDP Settings

In [ ]:
# MCDP SETTINGS
MCDP = False
MCDP_FOWARD_PASSES = 50
MCDP_DROPOUT = 0.5

## Functions

In [ ]:
def create_hook(embeddings_list):
    """Return a hook function that appends flattened outputs to embeddings_list."""
    def hook(module, input, output):
        output_np = output.detach().cpu().numpy()
        for x in output_np:
            embeddings_list.append(x.flatten())
    return hook

In [ ]:
def process_experiment(exp, mode, device, positive_threshold, batch_size):
    """
    Process a single experiment directory.
    
    Parameters:
        exp (str): Name of the experiment folder.
        mode (str): Either 'train' or 'test'.
        device (str): Device to run inference on.
        positive_threshold (float): Threshold for positive predictions.
        batch_size (int): Batch size for DataLoader.
        
    Returns:
        fold (str): Extracted fold name.
        result (dict): Dictionary containing outputs and optionally embeddings.
    """
    exp_path = os.path.join(BASE_EXPERIMENT_PATH, exp)
    model_path = os.path.join(exp_path, 'Model', 'best_model.pt')
    config_path = os.path.join(exp_path, 'Model', 'config.yaml')
    
    # Load configuration
    config = load_config(config_path)
    data_path = config['path_train'] if mode == 'train' else config['path_test']
    data_path = data_path.replace('/', '\\')  # Ensure platform independence
    
    # Extract the fold from the data path (platform independent)
    fold = os.path.normpath(data_path).split(os.sep)[-2]
    print(f"Processing {mode} data from: {data_path}")
    
    # Set up embeddings collection and hook handle (if needed)
    embeddings = []
    hook_handle = None

    # Choose model architecture and dataset based on experiment name
    if "NCNN" in exp:
        model_instance = NCNN()
        dataset = BaseDataset(model_name="NCNN", img_dir=data_path)
        #hook_handle = model_instance.merge_branch[0].register_forward_hook(create_hook(embeddings))
    elif "VGGFace" in exp:
        model_instance = VGGFace()
        dataset = BaseDataset(model_name="VGGFace", img_dir=data_path)
        # You may choose which layer to hook:
        # hook_handle = model_instance.VGGFace.features.conv5_3.register_forward_hook(create_hook(embeddings))
        #hook_handle = model_instance.VGGFace.classifier[3].register_forward_hook(create_hook(embeddings))
    elif "ViT" in exp:
        model_instance = ViT()
        dataset = BaseDataset(model_name="ViT", img_dir=data_path)
    elif "PainClassifier" in exp:
        model_instance = PainClassifier()
        dataset = BaseDataset(model_name="PainClassifier", img_dir=data_path)
    else:
        raise ValueError(f"Unknown experiment type in {exp}")

    dataloader = DataLoader(
        dataset, 
        batch_size=batch_size, 
        shuffle=False,
        pin_memory=True,
        num_workers=4
    )

    # Load model weights and prepare model for inference
    model_instance.load_state_dict(torch.load(model_path))
    model_instance = model_instance.to(device)
    model_instance.eval()

    # Accumulate outputs using lists (more efficient than repeated concatenation)
    probs_list, preds_list, logits_list, labels_list = [], [], [], []

    # If MCDP is activated, accumulate probabilities using a list
    if MCDP:
        probs_uq_list = []
        model_instance = MCDropout(model_instance, p=MCDP_DROPOUT)

    with torch.no_grad():
        for batch in tqdm(dataloader, desc=f"Processing {exp}"):
            inputs = batch['image'].to(device)
            labels = batch['label'].to(device)

            # If MCDP calculate probabilities for each forward pass
            if MCDP:
                probs = model_instance.predict(inputs, reps=MCDP_FOWARD_PASSES)
                preds = torch.ge(torch.mean(probs, dim=1), positive_threshold).type(torch.int)
                probs_uq_list.append(probs)

            else:
                logits = model_instance(inputs)
                probs = torch.sigmoid(logits)
                preds = (probs >= positive_threshold).int()
                logits_list.append(logits)

            probs_list.append(probs)
            preds_list.append(preds)
            labels_list.append(labels)

    # Concatenate tensors and convert to numpy arrays
    probs_all = torch.cat(probs_list).cpu().numpy()
    preds_all = torch.cat(preds_list).cpu().numpy()
    labels_all = torch.cat(labels_list).cpu().numpy()

    if MCDP:
    
        result = {
            'img_names': np.array(dataset.img_paths),
            'probs': probs_all.mean(axis=1),
            'preds': preds_all,
            'labels': labels_all,
            'probs_uq': probs_all
        }

    else:
        logits_all = torch.cat(logits_list).cpu().numpy()
        
        result = {
            'img_names': np.array(dataset.img_paths),
            'probs': probs_all,
            'preds': preds_all,
            'logits': logits_all,
            'labels': labels_all,
            #'embeddings': np.array(embeddings)
        }

    # Remove hook if it was set
    if hook_handle is not None:
        hook_handle.remove()

    # Cleanup GPU memory
    gc.collect()
    torch.cuda.empty_cache()

    return fold, result

In [ ]:
def main():
    """Main loop to process all experiments for each mode and save results."""
    modes = ['train', 'test']

    if MCDP:
        save_filenames = [f'train_results_MCDP_{MCDP_FOWARD_PASSES}_{MCDP_DROPOUT}.pkl', f'results_MCDP_{MCDP_FOWARD_PASSES}_{MCDP_DROPOUT}.pkl']
    else:
        save_filenames = ['train_results.pkl', 'results.pkl']
    
    for mode, save_filename in zip(modes, save_filenames):
        results = {'fold': []}
        # List all experiment directories in the base experiments path
        for exp in os.listdir(BASE_EXPERIMENT_PATH):
            # Filter out non-experiment files
            if any(sub in exp for sub in ['.pkl', 'masks', '.png']):
                continue
            try:
                fold, res = process_experiment(exp, mode, DEVICE, POSITIVE_THRESHOLD, BATCH_SIZE)
                #results[fold] = res
                for key, value in res.items():
                    if key not in results.keys():
                        results[key] = value.tolist()
                    else:
                        results[key].extend(value.tolist())
                
                results['fold'].extend([int(fold)] * len(res['img_names']))

            except Exception as e:
                print(f"Error processing {exp}: {e}")
                results['fold'].extend([fold] * len(res['img_names']))

        output_path = os.path.join(BASE_EXPERIMENT_PATH, save_filename)
        with open(output_path, 'wb') as f:
            pickle.dump(results, f)
        print(f"Saved {mode} results to {output_path}")

In [ ]:
main()

## mcdp & original


In [ ]:
train_calibrators = ["_LS_01", "_LS_03", "_LS_05", "_LINEAR", "_SIGMOID", "_STEP"]

model = "VGGFace"  # change model here
for calibrator in train_calibrators:
    print(f"Train calibrator: {calibrator}")
    MODEL_NAME = f'{model}{calibrator}'  
    BASE_EXPERIMENT_PATH = os.path.join('experiments', MODEL_NAME)
    main()
    

## Ensembles PKL

In [ ]:
# MCDP SETTINGS
MCDP = False

In [ ]:
EXPERIMENTS_FOLDER = 'ViT_B_32_ENSEMBLE_FINAL'  # change experiment folder here

for i in range(0, 10):
    print(f"Train ensemble {i}")
    MODEL_NAME = f'ensemble_{i}'  # change model here
    BASE_EXPERIMENT_PATH = os.path.join('experiments', EXPERIMENTS_FOLDER, MODEL_NAME)
    main()

In [ ]:
import os
import pickle
import numpy as np

# Path to the parent ensemble folder
PARENT_DIR = "experiments\\ViT_B_32_ENSEMBLE_FINAL"

#for ensembles_n in [3,5,10]:
for ensembles_n in [10]:
    print(f"Creating ensemble with {ensembles_n} models")
    save_filenames = [f'train_results.pkl', f'results.pkl']

    for save_filename in save_filenames:
        # 1. Gather all results.pkl paths
        paths = []
        for sub in sorted(os.listdir(PARENT_DIR)):
            p = os.path.join(PARENT_DIR, sub, save_filename)
            if os.path.isfile(p):
                paths.append(p)
        if not paths:
            raise RuntimeError(f"No results.pkl files found under {PARENT_DIR}")

        # 2. Load them
        all_probs = []
        for i, p in enumerate(paths[:ensembles_n]):
            with open(p, "rb") as f:
                d = pickle.load(f)
            # On first file, save the img_names and labels
            if i == 0:
                img_names = d["img_names"]
                labels    = d.get("labels", None)
                folds     = d.get("fold", None)
            else:
                # sanity check
                if d["img_names"] != img_names:
                    raise RuntimeError(f"Image ordering mismatch in {p}")
            all_probs.append(np.array(d["probs"]))

        # 3. Stack and average
        #    shape = (n_models, n_images)
        stacked = np.vstack(all_probs)
        mean_probs = stacked.mean(axis=0)

        # 4. Threshold to get predictions
        preds = (mean_probs >= 0.5).astype(int)

        # 5. Build the ensemble dict
        ensemble = {
            "fold":      folds,
            "img_names": img_names,
            "labels":    labels,
            "probs":     mean_probs.tolist(),
            # for each image, the list of its prob predictions across the 5 models
            "probs_uq":  [list(row) for row in stacked.T],
            "preds":     preds.tolist(),
        }

        # 6. Save it
        save_filename = f'ensemble_{ensembles_n}_{save_filename}'
        with open(os.path.join(PARENT_DIR, save_filename), "wb") as f:
            pickle.dump(ensemble, f)

        print(f"Wrote ensemble results to {save_filename}")


# Post-hoc Calibration

## Experiment Settings

In [ ]:
from utils.utils import create_folder
from calibration.calibrators import *

In [ ]:
BASE_EXPERIMENT_PATH = os.path.join('experiments', "ViT_B_32_ENSEMBLE")
MCDP = True

filename = 'ensemble_10_results.pkl'  # change here
filename_calib = 'ensemble_10_train_results.pkl'  # change here


positive_threshold = 0.5

if "MCDP" in filename:
    mcdp = "_MCDP_50_0.5"
    save_filename = f'results{mcdp}.pkl'

elif "ensemble" in filename:
    mcdp = "ensemble_10_"
    save_filename = f'{mcdp}results.pkl'


## Functions

In [ ]:
def calibrate_post_hoc(results, calib_results, calibrator):

    # Get unique folds from the results.
    unique_folds = results['fold'].unique()
    
    for fold in unique_folds:
        # Select calibration data for the current fold.
        calib_fold = calib_results[calib_results['fold'] == fold]
        # Fit the calibrator on this fold's calibration data.
        calibrator.fit(calib_fold['probs'].values, calib_fold['labels'].values)
        
        # Identify the indices for the current fold in the results.
        idx = results['fold'] == fold
        # Calibrate the probabilities for this fold.
        calibrated_probs = calibrator.predict(results.loc[idx, 'probs'].values)
        
        # Update the DataFrame in place.
        results.loc[idx, 'probs'] = calibrated_probs
        results.loc[idx, 'preds'] = (calibrated_probs >= positive_threshold).astype('float32')
    
    return results

## Platt Scaling

In [ ]:
create_folder(BASE_EXPERIMENT_PATH + '_PLATT')


with open(os.path.join(BASE_EXPERIMENT_PATH, filename), 'rb') as f:
    results = pd.DataFrame(pickle.load(f))

with open(os.path.join(BASE_EXPERIMENT_PATH, filename_calib), 'rb') as f:
    calib_results = pd.DataFrame(pickle.load(f))

calibrator = PlattScaling()

new_results = calibrate_post_hoc(results=results, calib_results=calib_results, calibrator=calibrator)

with open(os.path.join(BASE_EXPERIMENT_PATH + '_PLATT', save_filename), 'wb') as f:
    pickle.dump(new_results.to_dict('list'), f)

## Temperature Scaling

In [ ]:
create_folder(BASE_EXPERIMENT_PATH + '_TEMP')

with open(os.path.join(BASE_EXPERIMENT_PATH, filename), 'rb') as f:
    results = pd.DataFrame(pickle.load(f))

with open(os.path.join(BASE_EXPERIMENT_PATH, filename_calib), 'rb') as f:
    calib_results = pd.DataFrame(pickle.load(f))

calibrator = TemperatureScaling()

new_results = calibrate_post_hoc(results=results, calib_results=calib_results, calibrator=calibrator)

with open(os.path.join(BASE_EXPERIMENT_PATH + '_TEMP', save_filename), 'wb') as f:
    pickle.dump(new_results.to_dict('list'), f)

## Isotonic Regressor

In [ ]:
create_folder(BASE_EXPERIMENT_PATH + '_ISOTONIC')

with open(os.path.join(BASE_EXPERIMENT_PATH, filename), 'rb') as f:
    results = pd.DataFrame(pickle.load(f))

with open(os.path.join(BASE_EXPERIMENT_PATH, filename_calib), 'rb') as f:
    calib_results = pd.DataFrame(pickle.load(f))

calibrator = IsotonicRegressor()

new_results = calibrate_post_hoc(results=results, calib_results=calib_results, calibrator=calibrator)

with open(os.path.join(BASE_EXPERIMENT_PATH + '_ISOTONIC', save_filename), 'wb') as f:
    pickle.dump(new_results.to_dict('list'), f)

## Histogram Binning

In [ ]:
create_folder(BASE_EXPERIMENT_PATH + '_HIST')

with open(os.path.join(BASE_EXPERIMENT_PATH, filename), 'rb') as f:
    results = pd.DataFrame(pickle.load(f))

with open(os.path.join(BASE_EXPERIMENT_PATH, filename_calib), 'rb') as f:
    calib_results = pd.DataFrame(pickle.load(f))

calibrator = HistogramBinning()

new_results = calibrate_post_hoc(results=results, calib_results=calib_results, calibrator=calibrator)

with open(os.path.join(BASE_EXPERIMENT_PATH + '_HIST', save_filename), 'wb') as f:
    pickle.dump(new_results.to_dict('list'), f)